In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease/heart.csv


In [2]:
df = pd.read_csv('/kaggle/input/heart-disease/heart.csv')

In [3]:
df.sample(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
183,58,1,2,112,230,0,0,165,0,2.5,1,1,3,0
295,63,1,0,140,187,0,0,144,1,4.0,2,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
113,43,1,0,110,211,0,1,161,0,0.0,2,0,3,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
